In [1]:
import numpy as np
import pandas as pd


In [2]:
# Constants
GRID_SIZE = 5
NUM_STATES = GRID_SIZE * GRID_SIZE
NUM_ACTIONS = 4  # North, South, East, West
GAMMA = 0.9  # Discount factor


In [3]:
def create_distribution(n, temp=1.0):
    """
    Create a probability distribution over n elements.
    """
    logits = np.random.randn(n)
    exp_logits = np.exp(logits / temp)
    prob = exp_logits / exp_logits.sum()
    return prob


In [4]:
def create_MDP():
    """
    Create the Markov Decision Process (MDP) for the Gridworld.
    Returns:
        P: Transition probability matrix of shape (NUM_STATES, NUM_ACTIONS, NUM_STATES)
        R: Reward matrix of shape (NUM_STATES, NUM_ACTIONS)
    """
    P = np.zeros((NUM_STATES, NUM_ACTIONS, NUM_STATES))
    R = np.zeros((NUM_STATES, NUM_ACTIONS))
    
    # Special states and their corresponding transitions
    special_states = {
        1: {'next_state': 21, 'reward': 10},  # State A
        3: {'next_state': 13, 'reward': 5}    # State B
    }
    
    # Action effects: North, South, East, West
    action_effects = {
        0: (-1, 0),  # North
        1: (1, 0),   # South
        2: (0, 1),   # East
        3: (0, -1)   # West
    }

    for s in range(NUM_STATES):
        # Check if the state is a special state (A or B)
        if s in special_states:
            next_state = special_states[s]['next_state']
            reward = special_states[s]['reward']
            for a in range(NUM_ACTIONS):
                P[s, a, next_state] = 1.0
                R[s, a] = reward
        else:
            row, col = divmod(s, GRID_SIZE)
            for a in range(NUM_ACTIONS):
                delta_row, delta_col = action_effects[a]
                new_row = row + delta_row
                new_col = col + delta_col
                # Check if the new position is within grid boundaries
                if 0 <= new_row < GRID_SIZE and 0 <= new_col < GRID_SIZE:
                    next_state = new_row * GRID_SIZE + new_col
                    reward = 0  # Standard move reward
                else:
                    next_state = s  # Agent stays in the same state
                    reward = -1  # Penalty for hitting the wall
                P[s, a, next_state] = 1.0
                R[s, a] = reward
    return P, R


In [5]:
def initial_policy():
    """
    Create the equiprobable random policy.
    Returns:
        pi: Policy matrix of shape (NUM_STATES, NUM_ACTIONS)
    """
    pi = np.full((NUM_STATES, NUM_ACTIONS), 1.0 / NUM_ACTIONS)
    return pi


In [6]:
def policy_evaluation(P, R, pi, gamma):
    """
    Evaluate the given policy in the MDP.
    Args:
        P: Transition probability matrix
        R: Reward matrix
        pi: Policy matrix
        gamma: Discount factor
    Returns:
        v: State-value function vector of length NUM_STATES
    """
    # Compute the state transition matrix and reward vector under policy pi
    P_pi = np.einsum('sa,san->sn', pi, P)
    R_pi = np.einsum('sa,sa->s', pi, R)

    # Solve the Bellman equation: v = R_pi + gamma * P_pi * v
    # Rearranged to (I - gamma * P_pi) * v = R_pi
    I = np.eye(NUM_STATES)
    A = I - gamma * P_pi
    b = R_pi

    # Solve for v using linear system solver
    v = np.linalg.solve(A, b)
    return v


In [7]:
def policy_iteration(P, R, gamma):
    """
    Perform policy iteration to find the optimal policy.
    Args:
        P: Transition probability matrix
        R: Reward matrix
        gamma: Discount factor
    Returns:
        pi: Optimal policy matrix
        v: State-value function for the optimal policy
    """
    # Initialize policy arbitrarily
    pi = initial_policy()
    is_policy_stable = False

    while not is_policy_stable:
        # Policy Evaluation
        v = policy_evaluation(P, R, pi, gamma)
        is_policy_stable = True

        # Policy Improvement
        for s in range(NUM_STATES):
            old_action = np.argmax(pi[s])
            q_sa = np.zeros(NUM_ACTIONS)
            for a in range(NUM_ACTIONS):
                q_sa[a] = R[s, a] + gamma * np.dot(P[s, a], v)
            new_action = np.argmax(q_sa)
            if old_action != new_action:
                is_policy_stable = False
            # Update the policy to be greedy w.r.t. q_sa
            pi[s] = np.eye(NUM_ACTIONS)[new_action]
    return pi, v


In [8]:
def main():
    # Construct the MDP
    P, R = create_MDP()
    # Define the equiprobable random policy
    pi = initial_policy()
    # Evaluate the policy
    v = policy_evaluation(P, R, pi, GAMMA)
    # Reshape the value function for display purposes
    v_grid = v.reshape((GRID_SIZE, GRID_SIZE))
    # Create a pandas DataFrame for better display
    df = pd.DataFrame(np.round(v_grid, 1))
    df.index = [f"Row {i}" for i in range(GRID_SIZE)]
    df.columns = [f"Col {j}" for j in range(GRID_SIZE)]
    print("State-Value Function for the Equiprobable Random Policy:")
    print(df.to_string())

    # Optionally perform policy iteration to find the optimal policy
    # pi_opt, v_opt = policy_iteration(P, R, GAMMA)
    # v_opt_grid = v_opt.reshape((GRID_SIZE, GRID_SIZE))
    # df_opt = pd.DataFrame(np.round(v_opt_grid, 1))
    # df_opt.index = [f"Row {i}" for i in range(GRID_SIZE)]
    # df_opt.columns = [f"Col {j}" for j in range(GRID_SIZE)]
    # print("\nOptimal State-Value Function after Policy Iteration:")
    # print(df_opt.to_string())

if __name__ == "__main__":
    main()

State-Value Function for the Equiprobable Random Policy:
       Col 0  Col 1  Col 2  Col 3  Col 4
Row 0    3.3    8.8    4.4    5.3    1.5
Row 1    1.5    3.0    2.3    1.9    0.5
Row 2    0.1    0.7    0.7    0.4   -0.4
Row 3   -1.0   -0.4   -0.4   -0.6   -1.2
Row 4   -1.9   -1.3   -1.2   -1.4   -2.0
